<a href="https://colab.research.google.com/github/Mithara99/Automatic-Stage-Detection-of-Diabetic-Retinopathy/blob/main/CNNtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

In [13]:
google_drive_path = '/content/drive/MyDrive/Dataset_sorted'

#image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 32

#Create data generators for training, validation, and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of data will be used for validation
)

train_generator = train_datagen.flow_from_directory(
    os.path.join(google_drive_path, 'Train'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    os.path.join(google_drive_path, 'Validation'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(google_drive_path, 'Test'),
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

#CNN model for multiclass classification
num_classes = len(train_generator.class_indices)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Calculate steps_per_epoch and validation_steps based on the size of the datasets and batch size
steps_per_epoch = train_generator.samples // batch_size
validation_steps = validation_generator.samples // batch_size

# Ensure steps_per_epoch and validation_steps are at least 1
steps_per_epoch = max(1, steps_per_epoch)
validation_steps = max(1, validation_steps)

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_steps
)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test accuracy: {test_acc}")

# Generate predictions on the test set
y_pred = model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Compute classification report (includes precision, recall, f1-score)
class_report = classification_report(y_true, y_pred_classes, target_names=test_generator.class_indices.keys())
print("Classification Report:")
print(class_report)

Found 458 images belonging to 5 classes.
Found 19 images belonging to 5 classes.
Found 84 images belonging to 5 classes.
Epoch 1/30
14/14 [==============================] - 68s 5s/step - loss: 1.2189 - accuracy: 0.5235 - val_loss: 1.0843 - val_accuracy: 0.6842
Epoch 2/30
14/14 [==============================] - 61s 4s/step - loss: 0.8384 - accuracy: 0.7136 - val_loss: 0.5750 - val_accuracy: 0.7895
Epoch 3/30
14/14 [==============================] - 61s 4s/step - loss: 0.6692 - accuracy: 0.7512 - val_loss: 0.3778 - val_accuracy: 0.8421
Epoch 4/30
14/14 [==============================] - 63s 4s/step - loss: 0.6390 - accuracy: 0.7817 - val_loss: 0.5147 - val_accuracy: 0.8421
Epoch 5/30
14/14 [==============================] - 61s 4s/step - loss: 0.6348 - accuracy: 0.7488 - val_loss: 0.4873 - val_accuracy: 0.8421
Epoch 6/30
14/14 [==============================] - 62s 4s/step - loss: 0.6013 - accuracy: 0.7512 - val_loss: 0.4857 - val_accuracy: 0.7895
Epoch 7/30
14/14 [=====================

ValueError: Number of classes, 4, does not match size of target_names, 5. Try specifying the labels parameter